Test the trainMLP.py codes.

In [1]:
import sys
from random import shuffle
import argparse

import numpy as np
import scipy.io

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils import np_utils, generic_utils

from sklearn import preprocessing
from sklearn.externals import joblib

from spacy.en import English

from features import get_questions_matrix_sum, get_images_matrix, get_answers_matrix
from utils import grouper, selectFrequentAnswers

Using TensorFlow backend.


In [2]:
questions_train = open('../data/preprocessed/questions_train2014.txt', 'r').read().decode('utf8').splitlines()
answers_train = open('../data/preprocessed/answers_train2014_modal.txt', 'r').read().decode('utf8').splitlines()
images_train = open('../data/preprocessed/images_train2014.txt', 'r').read().decode('utf8').splitlines()
vgg_model_path = '../features/coco/vgg_feats.mat'


In [3]:
idx = 1

print len(questions_train)
print len(answers_train)
print len(images_train)

print questions_train[idx]
print answers_train[idx]
print images_train[idx]


248349
248349
248349
Is there a shadow?
yes
487025


In [4]:
parser = argparse.ArgumentParser()
parser.add_argument('-num_hidden_units', type=int, default=1024)
parser.add_argument('-num_hidden_layers', type=int, default=3)
parser.add_argument('-dropout', type=float, default=0.5)
parser.add_argument('-activation', type=str, default='tanh')
parser.add_argument('-language_only', type=bool, default= False)
parser.add_argument('-num_epochs', type=int, default=1)
parser.add_argument('-model_save_interval', type=int, default=10)
parser.add_argument('-batch_size', type=int, default=128)
parser.add_argument('-f', type=str, default='')
args = parser.parse_args()
print args

Namespace(activation='tanh', batch_size=128, dropout=0.5, f='/Users/jyang/Library/Jupyter/runtime/kernel-a9934ce4-7cd6-4817-8ae6-db9c3ec37332.json', language_only=False, model_save_interval=10, num_epochs=1, num_hidden_layers=3, num_hidden_units=1024)


In [5]:
maxAnswers = 1000
questions_train, answers_train, images_train = \
    selectFrequentAnswers(questions_train,answers_train,images_train, maxAnswers)

In [6]:
#encode the remaining answers
labelencoder = preprocessing.LabelEncoder()
labelencoder.fit(answers_train)
nb_classes = len(list(labelencoder.classes_))
print 'nb_classes:', nb_classes
joblib.dump(labelencoder,'../models/labelencoder.pkl')

nb_classes: 1000


['../models/labelencoder.pkl', '../models/labelencoder.pkl_01.npy']

In [7]:
features_struct = scipy.io.loadmat(vgg_model_path)
VGGfeatures = features_struct['feats']
print 'loaded vgg features'
image_ids = open('../features/coco_vgg_IDMap.txt').read().splitlines()
id_map = {}
for ids in image_ids:
    id_split = ids.split()
    id_map[id_split[0]] = int(id_split[1])

loaded vgg features


In [8]:
len(id_map)
id_map.items()[:10]

[('378466', 58761),
 ('378467', 27080),
 ('287140', 82608),
 ('376426', 111769),
 ('378461', 92750),
 ('13356', 73748),
 ('258823', 116456),
 ('370250', 98962),
 ('370252', 48071),
 ('89378', 29771)]

In [9]:
nlp = English()
print 'loaded word2vec features...'
img_dim = 4096
word_vec_dim = 300

model = Sequential()
if args.language_only:
    model.add(Dense(args.num_hidden_units, input_dim=word_vec_dim, init='uniform'))
else:
    model.add(Dense(args.num_hidden_units, input_dim=img_dim+word_vec_dim, init='uniform'))
model.add(Activation(args.activation))
if args.dropout>0:
    model.add(Dropout(args.dropout))
for i in xrange(args.num_hidden_layers-1):
    model.add(Dense(args.num_hidden_units, init='uniform'))
    model.add(Activation(args.activation))
    if args.dropout>0:
        model.add(Dropout(args.dropout))
model.add(Dense(nb_classes, init='uniform'))
model.add(Activation('softmax'))

json_string = model.to_json()
if args.language_only:
    model_file_name = '../models/mlp_language_only_num_hidden_units_' + \
    str(args.num_hidden_units) + '_num_hidden_layers_' + str(args.num_hidden_layers)
else:
    model_file_name = '../models/mlp_num_hidden_units_' + str(args.num_hidden_units) + \
    '_num_hidden_layers_' + str(args.num_hidden_layers)
open(model_file_name  + '.json', 'w').write(json_string)

loaded word2vec features...


In [10]:
from pprint import pprint
pprint(json_string.replace('null', ''))

'{"class_name": "Sequential", "keras_version": "1.1.0", "config": [{"class_name": "Dense", "config": {"W_constraint": , "b_constraint": , "name": "dense_1", "output_dim": 1024, "activity_regularizer": , "trainable": true, "init": "uniform", "bias": true, "input_dtype": "float32", "input_dim": 4396, "b_regularizer": , "W_regularizer": , "activation": "linear", "batch_input_shape": [, 4396]}}, {"class_name": "Activation", "config": {"activation": "tanh", "trainable": true, "name": "activation_1"}}, {"class_name": "Dropout", "config": {"p": 0.5, "trainable": true, "name": "dropout_1"}}, {"class_name": "Dense", "config": {"W_constraint": , "b_constraint": , "name": "dense_2", "activity_regularizer": , "trainable": true, "init": "uniform", "bias": true, "input_dim": , "b_regularizer": , "W_regularizer": , "activation": "linear", "output_dim": 1024}}, {"class_name": "Activation", "config": {"activation": "tanh", "trainable": true, "name": "activation_2"}}, {"class_name": "Dropout", "config":

In [11]:
print 'Compiling model...'
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
print 'Compilation done...'

Compiling model...
Compilation done...


In [12]:
print model

In [13]:
print 'Training started...'
for k in xrange(args.num_epochs):
    #shuffle the data points before going through them
    index_shuf = range(len(questions_train))
    shuffle(index_shuf)
    questions_train = [questions_train[i] for i in index_shuf]
    answers_train = [answers_train[i] for i in index_shuf]
    images_train = [images_train[i] for i in index_shuf]
    progbar = generic_utils.Progbar(len(questions_train))
    for qu_batch,an_batch,im_batch in zip(grouper(questions_train, args.batch_size, fillvalue=questions_train[-1]), 
                                          grouper(answers_train, args.batch_size, fillvalue=answers_train[-1]), 
                                          grouper(images_train, args.batch_size, fillvalue=images_train[-1])):
        X_q_batch = get_questions_matrix_sum(qu_batch, nlp)
        if args.language_only:
            X_batch = X_q_batch
        else:
            X_i_batch = get_images_matrix(im_batch, id_map, VGGfeatures)
            X_batch = np.hstack((X_q_batch, X_i_batch))
        Y_batch = get_answers_matrix(an_batch, labelencoder)
        loss = model.train_on_batch(X_batch, Y_batch)
        progbar.add(args.batch_size, values=[("train loss", loss)])
    #print type(loss)
    if k%args.model_save_interval == 0:
        model.save_weights(model_file_name + '_epoch_{:02d}.hdf5'.format(k))

model.save_weights(model_file_name + '_epoch_{:02d}.hdf5'.format(k))

Training started...
215424/215375 [==============================] - 389s - train loss: 4.3101   
